In [1]:
fp = ('/Users/cmutel/Box Sync/LC-Impact (Chris Mutel)/12-water consumption/spatial layers EQ'
      '/shapefiles/CF_CORE_plants_noVS_and_animals_inclVS_noCpA_Option3_SW_PDF_perm3.tif')

fp = ('/Users/cmutel/Box Sync/LC-Impact (Chris Mutel)/12-water consumption/spatial layers EQ'
      '/shapefiles/CF_waterConsumption_extended_plants_NoVS_animalsInclVS_noCPA_Option3_PDF_perm3.tif')

In [2]:
%matplotlib inline

In [8]:
import rasterio
from scipy.cluster.vq import kmeans, vq
import numpy as np
import matplotlib.pyplot as plt
from pandarus import convert_to_vector

In [4]:
with rasterio.open(fp) as f:
    profile = f.profile
    array = f.read(1, masked=True)

In [5]:
profile.update(
    driver='GTiff',
    count=1,
    compress='lzw',
    nodata=-1,
    dtype=np.float32,
    tiled=False,
)
profile.pop('blockysize', None)
profile.pop('blockxsize', None)

array[array < 0] = -1
array = array.astype(np.float32)

In [7]:
raw_flat = array.data.flatten()
flat = np.log(raw_flat)
mask = np.isnan(flat)
flat = flat[~mask]

def backwards(vector):
    a = raw_flat.copy()
    a[~mask] = vector
    return a.reshape(*array.shape)

# In remaining subplots add k-means classified images
for i in (10,): # range(5, 15):
    print("Calculate k-means with ", i, " cluster.")
    
    centroids, variance = kmeans(flat, i)
    code, distance = vq(flat, centroids)    
    codeim = backwards(code)

    for value in np.unique(codeim):
        array[codeim == value] = np.median(array[codeim == value])
    
    with rasterio.Env():
        with rasterio.open('water_eq_extended.{}.tif'.format(i), 'w', **profile) as dst:
            dst.write(array.data, 1)

/Users/cmutel/miniconda3/envs/pandarus/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in log
  from ipykernel import kernelapp as app


Calculate k-means with  10  cluster.


/Users/cmutel/.local/lib/python3.5/site-packages/rasterio/__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


In [9]:
convert_to_vector("water_eq_extended.10.tif")

/Users/cmutel/.local/lib/python3.5/site-packages/rasterio/rio/helpers.py:55: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  features = list(collection())
/Users/cmutel/.local/lib/python3.5/site-packages/rasterio/features.py:102: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


'/Users/cmutel/Library/Application Support/pandarus/raster-conversion/e4630c5b46f9061f20e47d5cf49bde7ef8a58ebddf344e9aa132f8dd9fce18f3.1.geojson'